## Combine relevant Datasets

The **Occurence Dataset** in **/cleaned_data/gbif_occurences_cleaned.csv** and the **Dive Site Dataset** in /cleaned_data/dive_sites_data.xlsx will be combined in this script.

This is done using the location data.

In [1]:
import pandas as pd

# Load the occurences dataset
occurrences = pd.read_csv(r"cleaned_data\gbif_occurences_cleaned.csv")

# Load the dive sites excel file
dive_sites_excel_file = pd.ExcelFile(r"cleaned_data\dive_sites_data.xlsx")

dive_sites = pd.read_excel(dive_sites_excel_file, sheet_name='Dive Sites')
dive_types = pd.read_excel(dive_sites_excel_file, sheet_name='Dive Types Connections')
sightings = pd.read_excel(dive_sites_excel_file, sheet_name='Common Sightings Connections')

# Rename columns of the dive site dataset to match the occurences dataset
dive_sites = dive_sites.rename(columns={'Latitude': 'decimalLatitude', 'Longitude': 'decimalLongitude'})

# Print out first rows of the occurences dataset
print("Occurence Dataset")
print(occurrences.head())

# Print out first rows of the dive sites dataset
print("Dive Sites Dataset")
print(dive_sites.head())
print(len(dive_sites))



Occurence Dataset
   decimalLatitude  decimalLongitude                           scientificName  \
0          12.3180          -69.1506         Myripristis jacobus Cuvier, 1829   
1          12.3180          -69.1506     Diplectrum formosum (Linnaeus, 1766)   
2          12.3180          -69.1506         Caretta caretta (Linnaeus, 1758)   
3          12.3180          -69.1506  Aulostomus maculatus Valenciennes, 1841   
4          12.2187          -69.0857         Myripristis jacobus Cuvier, 1829   

           eventDate       dateIdentified  depth            commonName  
0  2014-08-19T10:26Z  2014-08-19T00:00:00   9.60  Blackbar soldierfish  
1  2014-08-19T10:26Z  2014-08-19T00:00:00   9.60            Sand perch  
2  2014-08-19T10:26Z  2014-08-19T00:00:00   9.60            Loggerhead  
3  2014-08-19T10:26Z  2014-08-19T00:00:00   9.60  Atlantic trumpetfish  
4  2014-08-16T12:57Z  2014-08-16T00:00:00  11.75  Blackbar soldierfish  
Dive Sites Dataset
   ID  decimalLatitude                

### Merge

Merge the two datasets based on their geographical proximity.
We'll consider a small **threshold of 0.01 degrees** (=1 km) to match occurrences to nearby dive sites. 

We use **cKDTree** because it has nlogn complexity compared to a brute force n squared.

We use the **euclidean distance** to calculate if the occurence is close enough to a dive spot.


In [ ]:
from scipy.spatial import cKDTree
import numpy as np

# Create new Columns in the dive site dataframe: Occurences and Distances_to_occurences
# The Occurences column includes an array of all Common names of the species that were found at the dive site
# Every Occurence that is closer than the threshold is included in the Occurences column
# The Distances_to_occurences column includes an array of all distances to the occurences

dive_sites['Occurences'] = None
# dive_sites['Occurence Location'] = None           # for manual check
dive_sites['Distances_to_occurences'] = None
dive_sites['Dive Types'] = None
dive_sites['Common Sightings'] = None

print(f"🟢 Start merging dive sites with occurrences ...")

# Convert latitude and longitude columns to numeric types
occurrences['decimalLatitude'] = pd.to_numeric(occurrences['decimalLatitude'], errors='coerce')
occurrences['decimalLongitude'] = pd.to_numeric(occurrences['decimalLongitude'], errors='coerce')
dive_sites['decimalLatitude'] = pd.to_numeric(dive_sites['decimalLatitude'], errors='coerce')
dive_sites['decimalLongitude'] = pd.to_numeric(dive_sites['decimalLongitude'], errors='coerce')

# Create coordinate arrays for both datasets
occurrence_coords = occurrences[['decimalLatitude', 'decimalLongitude']].values
dive_sites_coords = dive_sites[['decimalLatitude', 'decimalLongitude']].values

# Create a KDTree for occurrences
occurrence_tree = cKDTree(occurrence_coords)
threshold = 0.01

print(f" Finding occurences for {len(dive_sites_coords)} dive sites ...")
print(f"🔍 Using a threshold of {threshold} degrees ...")

# For every dive site ...
for i, (lat, lon) in enumerate(dive_sites_coords):
    
    # ... print out progress
    print(f"  {i}/{len(dive_sites_coords)} dive sites processed ...")

    # ... query the KDTree to find all occurrences within the threshold
    indices = occurrence_tree.query_ball_point([lat, lon], r=threshold)
    
    # ... get the common names and euclidean distances for these occurrences
    common_names = occurrences.iloc[indices]['commonName'].tolist()
    distances = np.sqrt((occurrences.iloc[indices]['decimalLatitude'] - lat)**2 +
                        (occurrences.iloc[indices]['decimalLongitude'] - lon)**2).tolist()
    
    # ... update the dive sites with the results
    dive_sites.at[i, 'Occurences'] = common_names
    # dive_sites.at[i, 'Occurence Locations'] = occurrences.iloc[indices][['decimalLatitude', 'decimalLongitude']].values   # for manual check
    dive_sites.at[i, 'Distances_to_occurences'] = distances

# Create a Column Dive Types for each dive site
for i, row in dive_sites.iterrows():
    dive_site_id = row['ID']
    dive_types_names = dive_types[dive_types['ID'] == dive_site_id]['Dive Types'].values
    dive_sites.at[i, 'Dive Types'] = dive_types_names

# Create a Column Common Sighting for each dive site
for i, row in dive_sites.iterrows():
    dive_site_id = row['ID']
    sightings_names = sightings[sightings['ID'] == dive_site_id]['Common Sightings'].values
    dive_sites.at[i, 'Common Sightings'] = sightings_names

# Save the updated dive sites DataFrame
output_path = r"final_data\dive_sites_with_occurrences.xlsx"
dive_sites.to_excel(output_path, index=False)

# Print out first rows of the updated dive site dataset
print(f"💾 Merged Dataset saved to {output_path}")
print(dive_sites.head())

🟢 Start merging dive sites with occurrences ...
 Finding occurences for 4395 dive sites ...
🔍 Using a threshold of 0.01 degrees ...
  0/4395 dive sites processed ...
  1/4395 dive sites processed ...
  2/4395 dive sites processed ...
  3/4395 dive sites processed ...
  4/4395 dive sites processed ...
  5/4395 dive sites processed ...
  6/4395 dive sites processed ...
  7/4395 dive sites processed ...
  8/4395 dive sites processed ...
  9/4395 dive sites processed ...
  10/4395 dive sites processed ...
  11/4395 dive sites processed ...
  12/4395 dive sites processed ...
  13/4395 dive sites processed ...
  14/4395 dive sites processed ...
  15/4395 dive sites processed ...
  16/4395 dive sites processed ...
  17/4395 dive sites processed ...
  18/4395 dive sites processed ...
  19/4395 dive sites processed ...
  20/4395 dive sites processed ...
  21/4395 dive sites processed ...
  22/4395 dive sites processed ...
  23/4395 dive sites processed ...
  24/4395 dive sites processed ...
  2

KeyError: 'Dive Site ID'